# What to do now :

Make a Portfolio using the strategy given. Test the strategy on a historical dataset. Evaluate the performance of the strategy using metrics such as Sharpe Ratio, etc. Make graphical representation.

Note: We are using data for a day.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# No Need to download data again
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
# print(t)
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
tickers=[]
for stock_group in t:
    for stock_name in stock_group:
        tickers.append(stock_name)
    
tickers


['A',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APO',
 'APTV',
 'ARE',
 'ATO',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXON',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BG',
 'BIIB',
 'BK',
 'BKNG',
 'BKR',
 'BLDR',
 'BLK',
 'BMY',
 'BR',
 'BRK.B',
 'BRO',
 'BSX',
 'BX',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COIN',
 'COO',
 'COP',
 'COR',
 'COST',
 'CPAY',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CRWD',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTRA',
 'CTSH',
 'CTVA',


## STRATEGIES ------------------------------------------------------------------------ START

In [3]:
from scipy import stats
                                
def strat_per(stock, max_weight=0.08, momentum_days=20, reversal_days=5):
    """
    Simpler strategy combining short-term reversal with medium-term momentum
    """
    dfs = []
    n = None

    for ticker in stock:
        df = pd.read_csv(f"sp500_csvs/{ticker}.csv")
        df.reset_index(drop=True, inplace=True)
        df['Close'] = pd.to_numeric(df['Close'])
        dfs.append(df)
        if n is None:
            n = len(df)

    portfolio = np.zeros((len(stock), n))

    for i, df in enumerate(dfs):
        returns = df['Close'].pct_change()
        
        # Medium-term momentum
        momentum = returns.rolling(window=momentum_days).mean()
        
        # Short-term reversal
        short_term_return = returns.rolling(window=reversal_days).mean()
        reversal = -short_term_return  # Opposite of recent performance
        
        # Combine: follow medium-term trend but fade short-term moves
        signal = 0.7 * momentum + 0.3 * reversal # Try changing parameters. Might get some better results 
        
        # Volatility adjustment
        vol = returns.rolling(window=20).std()
        vol_adjusted_signal = signal / (vol + 1e-8) # Avoid division by zero. Important.
        
        portfolio[i, :] = vol_adjusted_signal.shift(1).fillna(0).to_numpy()

    abs_sum = np.sum(np.abs(portfolio), axis=0, keepdims=True)
    abs_sum[abs_sum == 0] = 1.0 # Prevents division by 0
    normalized_portfolio = portfolio / abs_sum # Normalising the portfolio to have a sum of 1
    clipped_portfolio = np.clip(normalized_portfolio, -max_weight, max_weight) # Clip to prevent over-weighting
    market_neutral_portfolio = clipped_portfolio - clipped_portfolio.mean(axis=0, keepdims=True) # Subtract the mean to make it market neutral
    
    return market_neutral_portfolio


## Assignment 
Try creating your own strategies. Just remember, input can be anything ,but , output should be a portfolio with rows as stocks and columns as dates. See example output if you don't understand.
Don't worry if this doesn't correspond to the other backtester. This is because the normalization you might be doing here is different than than how the other does that.

P.S. Before you backtest , make sure to complete the backtester : backtest_portfolio () function. 

In [ ]:
def your_strat_1(stocks, window=20, num_std=1, max_weight=0.08):
    """
    Buy when price crosses below lower band, sell when above upper band.
    """
    close_df = pd.DataFrame({ticker: pd.read_csv(f"sp500_csvs/{ticker}.csv")['Close'].astype(float).values for ticker in stocks})

    rolling_mean = close_df.rolling(window=window, min_periods=1).mean()
    rolling_std = close_df.rolling(window=window, min_periods=1).std()
    upper_band = rolling_mean + num_std * rolling_std
    lower_band = rolling_mean - num_std * rolling_std
    # Generate signals: 1 for buy, -1 for sel; 0 for hold
    signal = (close_df < lower_band).astype(int) - (close_df > upper_band).astype(int)
    signal = signal.shift(1).fillna(0)

    # Normalize weights daily
    abs_sum = signal.abs().sum(axis=1).replace(0, 1)
    normalized = signal.div(abs_sum, axis=0)
    clipped = normalized.clip(-max_weight, max_weight)
    market_neutral = clipped.sub(clipped.mean(axis=1), axis=0)
    return market_neutral.T.values

In [ ]:
def your_strat_2(stocks, window=5, threshold=2, max_weight=0.05):
    """
    Volatility Breakout Strategy (Vectorized):
    If today's High-Low range is much larger than average, take a position in the direction of the breakout.
    """
    high_df = pd.DataFrame({ticker: pd.read_csv(f"sp500_csvs/{ticker}.csv")['High'].astype(float).values for ticker in stocks})
    low_df = pd.DataFrame({ticker: pd.read_csv(f"sp500_csvs/{ticker}.csv")['Low'].astype(float).values for ticker in stocks})
    close_df = pd.DataFrame({ticker: pd.read_csv(f"sp500_csvs/{ticker}.csv")['Close'].astype(float).values for ticker in stocks})

    range_df = high_df - low_df
    avg_range = range_df.rolling(window=window, min_periods=1).mean()
    breakout = range_df > (threshold * avg_range)

    # Direction: 1 if close > previous high, -1 if close < previous low

    direction = (close_df > high_df.shift(1)).astype(int) - (close_df < low_df.shift(1)).astype(int)
    signal = direction.where(breakout, 0)
    signal = signal.shift(1).fillna(0)
    
    # Normalize weights
    abs_sum = signal.abs().sum(axis=1).replace(0, 1)
    normalized = signal.div(abs_sum, axis=0)
    clipped = normalized.clip(-max_weight, max_weight)
    market_neutral = clipped.sub(clipped.mean(axis=1), axis=0)
    return market_neutral.T.values

In [ ]:
def your_strat_3(stocks, window=20, vol_window=30, max_weight=0.08):
    """
    - If volatility is high, mean reversion signals are stronger.
    - If volatility is low, mean reversion signals are weaker.
    """
    close_df = pd.DataFrame({
        ticker: pd.read_csv(f"sp500_csvs/{ticker}.csv")['Close'].astype(float).values
        for ticker in stocks
    })
    rolling_mean = close_df.rolling(window=window, min_periods=1).mean()
    rolling_std = close_df.rolling(window=window, min_periods=1).std().replace(0, 1)
    zscore = (close_df - rolling_mean) / rolling_std

    returns = close_df.pct_change().fillna(0)
    vol = returns.rolling(window=vol_window, min_periods=1).std()

    norm_vol = (vol - vol.min()) / (vol.max() - vol.min())
    regime_factor = 0.5 + 1.5 * norm_vol

    signal = -zscore * regime_factor
    signal = signal.shift(1).fillna(0)

    # Normalize weights daily
    abs_sum = signal.abs().sum(axis=1).replace(0, 1)
    normalized = signal.div(abs_sum, axis=0)
    clipped = normalized.clip(-max_weight, max_weight)
    market_neutral = clipped.sub(clipped.mean(axis=1), axis=0)
    return market_neutral.T.values


## STRATEGIES ------------------------------------------------------ END

In [26]:
print("Testing the strat......\n")
portfolio = your_strat_3(tickers)
# print(portfolio.shape)
print(portfolio)  # Show first 10 elements
print(np.sum(portfolio))   # Sum of allocations : should be ideally 0 after Market Neutralisation

Testing the strat......

[[ 0.00000000e+00  0.00000000e+00  1.69123061e-03 ... -1.49472096e-03
  -1.34456684e-03 -1.15792006e-03]
 [ 0.00000000e+00  0.00000000e+00  1.06585811e-03 ...  1.89940774e-03
   1.90360652e-03  1.90699383e-03]
 [ 0.00000000e+00  0.00000000e+00  1.06585811e-03 ...  3.05405289e-04
   2.47179174e-04  2.92474657e-04]
 ...
 [ 0.00000000e+00  0.00000000e+00  1.06585811e-03 ... -1.17075304e-03
  -9.91836626e-04 -1.35620914e-03]
 [ 0.00000000e+00  0.00000000e+00  1.07204515e-03 ... -2.01622758e-03
  -1.76690743e-03 -1.40039653e-03]
 [ 0.00000000e+00  0.00000000e+00 -2.65144220e-03 ... -4.95166860e-05
  -2.82904923e-04 -2.62407125e-04]]
1.0158540675320182e-14


/tmp/ipykernel_62655/945693782.py:17: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [27]:
print(portfolio.shape)

(503, 1006)


# Studying the metrics

The strategy was written in such a way that we didn't need to make a portfolio. 

All we had to do is just call the strat and the portfolio would take care of itself.

We have the portfolio, which is what the person's standing for the day is.

How much percentage of his portfolio is each stock in? So, basically we know the no. of stocks he owns for the day, given the portfolio.

So what do we need to do now ?

- As a day passes , we need to update the portfolio. ( We can also assume the portfolio to be constant for simplicity but we won't )

            PORTFOLIO_VAL_NEW  = PORTFOLIO_VAL_OLD + PnL
        
 We can also clip it for saftey.


- Calculate metrics as needed :
    
    - Total PnL

                    PnL = Summ ( ( Close_new- Close_old ) * Quantity ) for all stocks in the portfolio
    
    - Cumilitive Returns

                  Cumulative Return=( ∏ ( 1 + r_t ) ) − 1  : r_t daily return

    - Sharpe Ratio 

                    Sharpe Ratio = ( PnL - Risk_free ) / Volatility

        aka

                    Sharpe Ration = (Mean(Ret_daily) / Std(Ret_daily)) * sqrt(252)

    - Maximum Drawdown

                    Max Drawdown = max { ( V_peak - V_trough ) / V_peak } 
                    V :  portfolio value at time t = Sum ( weight of stock * Price )


    - Turnover

                    Turnover_at_t = Sum (( | weight_new - weight_old | )) : for all stocks in the portfolio

                    Turnover = Sum ( Turnover_at_t ) / days : for all days in the portfolio


    - Correlation : A metric introduced by me. It is the correlation between the stock and the portfolio. We'll measure it in time shift. 

                    Corr(  delta(close*volume,1) , PnL created by Portfolio  )  


- Plotting the metrics : ) 

Note : we have used a lot of assumptions while calculating the metrics and creating this backtester. For example : we have assumed that he player's influence in the market is negligible. 

In [28]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as py

def backtest_portfolio(portfolio, tickers, risk_free_rate=0.0451):
    n_stocks, n_days = portfolio.shape
    price_matrix = np.zeros((n_stocks, n_days))

    # Load price data
    for i, ticker in enumerate(tickers):
        df = pd.read_csv(f"sp500_csvs/{ticker}.csv")
        df = df.reset_index(drop=True)
        df['Close'] = pd.to_numeric(df['Close'])
        price_matrix[i, :] = df['Close'].to_numpy()

    # # Stock daily returns
    stock_returns = pd.DataFrame(price_matrix).pct_change(axis=1).fillna(0).values

    # print(stock_returns)

    # Portfolio returns
    daily_returns = np.nansum(portfolio * stock_returns, axis=0)

    # Cumulative Returns
    cumulative_returns = np.cumprod(1 + daily_returns)

    # Sharpe Ratio
    excess_returns = daily_returns  # Optionally subtract risk-free rate per period
    sharpe_ratio = (np.mean(excess_returns) / np.std(excess_returns)) * np.sqrt(252) if np.std(excess_returns) > 0 else 0

    # Max Drawdown
    cumulative_curve = np.cumprod(1 + daily_returns)
    rolling_max = np.maximum.accumulate(cumulative_curve)
    drawdown = 1 - cumulative_curve / rolling_max
    max_drawdown = np.max(drawdown)

    # Turnover
    turnover = np.mean(np.sum(np.abs(np.diff(portfolio, axis=1)), axis=0))

    # Mean Correlation with individual stocks
    correlations = []
    for i in range(n_stocks):
        x = daily_returns
        y = stock_returns[i, :]
        mask = ~np.isnan(x) & ~np.isnan(y)
        if np.sum(mask) > 1:
            corr = np.corrcoef(x[mask], y[mask])[0, 1]
            correlations.append(corr)
        else:
            correlations.append(np.nan)
    mean_correlation = np.nanmean(correlations)

    results = {
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "Turnover": turnover,
        "Total Return": cumulative_returns[-1] * 100,
        "Mean Correlation (Portfolio vs Stocks)": mean_correlation,
        "Daily Returns": daily_returns,
        "Cumulative Returns": cumulative_returns
    }

    # PLotting 
    days = np.arange(n_days)

    # Create subplots layout: 3 rows for returns, drawdown, top stocks
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=("Cumulative Returns", "Daily Returns", "Drawdown Curve", "Portfolio vs Top 5 Stocks"),
        vertical_spacing=0.1,
        shared_xaxes=True
    )

    # Cumulative Returns
    fig.add_trace(go.Scatter(x=days, y=cumulative_returns, name="Cumulative Return"), row=1, col=1)

    # Daily Returns
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Daily Return"), row=2, col=1)

    # Drawdown
    fig.add_trace(go.Scatter(x=days, y=drawdown, name="Drawdown"), row=3, col=1)

    # Portfolio vs Top 5 correlated stocks
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Portfolio"), row=4, col=1)
    top_5_idx = np.argsort(correlations)[-5:]

    # for i in top_5_idx:
    #     print(stock_returns[i])
    # print(top_5_idx)

    for i in top_5_idx:
        fig.add_trace(go.Scatter(x=days, y=stock_returns[i], name=f"{tickers[i]}"), row=4, col=1)

    # Layout
    fig.update_layout(
        height=1200,
        title_text="Portfolio Performance Summary",
        showlegend=True
    )

    # Save and open in browser
    py.plot(fig, filename="portfolio_summary.html")

    return results, correlations


In [29]:
results,corr = backtest_portfolio(portfolio, tickers)

for key, value in results.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        print(f"{key}: {value:.4f}")

# print(corr)

counter = 0
for x in corr:
    if x>0:
        counter+=1

print("No. of correlating stocks : "+ str(counter))

/tmp/ipykernel_62655/3760714932.py:19: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Sharpe Ratio: 0.4629
Max Drawdown: 0.0979
Turnover: 0.3843
Total Return: 112.6024
Mean Correlation (Portfolio vs Stocks): 0.2044
No. of correlating stocks : 492


In [30]:
# Test all strategies and print their results
print('Testing Bollinger Bands Strategy (your_strat_1)')
portfolio1 = your_strat_1(tickers)
results1, corr1 = backtest_portfolio(portfolio1, tickers)
print('Results for your_strat_1:')
for key, value in results1.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        print(f"{key}: {value:.4f}")
print("\n")

print('Testing Volatility Breakout Strategy (your_strat_2)')
portfolio2 = your_strat_2(tickers)
results2, corr2 = backtest_portfolio(portfolio2, tickers)
print('Results for your_strat_2:')
for key, value in results2.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        print(f"{key}: {value:.4f}")
print("\n")

print('Testing Price-Volume Trend Strategy (your_strat_3)')
portfolio3 = your_strat_3(tickers)
results3, corr3 = backtest_portfolio(portfolio3, tickers)
print('Results for your_strat_3:')
for key, value in results3.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        print(f"{key}: {value:.4f}")
print("\n")

Testing Bollinger Bands Strategy (your_strat_1)


/tmp/ipykernel_62655/3760714932.py:19: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Results for your_strat_1:
Sharpe Ratio: 0.4891
Max Drawdown: 0.1086
Turnover: 0.6486
Total Return: 114.8000
Mean Correlation (Portfolio vs Stocks): 0.2195


Testing Volatility Breakout Strategy (your_strat_2)


/tmp/ipykernel_62655/3760714932.py:19: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Results for your_strat_2:
Sharpe Ratio: 0.6807
Max Drawdown: 0.0450
Turnover: 0.5459
Total Return: 115.2178
Mean Correlation (Portfolio vs Stocks): -0.0340


Testing Price-Volume Trend Strategy (your_strat_3)


/tmp/ipykernel_62655/945693782.py:17: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/tmp/ipykernel_62655/3760714932.py:19: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

/home/rhythm-kedia/Projects/SOC/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Results for your_strat_3:
Sharpe Ratio: 0.4629
Max Drawdown: 0.0979
Turnover: 0.3843
Total Return: 112.6024
Mean Correlation (Portfolio vs Stocks): 0.2044


